In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_athens = pd.read_csv('data_athens.csv')

In [3]:
data_athens.corr()

,Unnamed: 0,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
Unnamed: 0,1.000000,0.007060,0.018539,-0.018551,-0.020357,-0.008189,-0.016446,0.024422,0.144658,-0.020244,-0.064565,0.003949,-0.022415,-0.014750,-0.061011,-0.008528
realSum,0.007060,1.000000,0.091463,-0.030984,0.086842,0.041307,0.037163,0.080990,-0.112416,-0.052813,0.127751,0.107934,-0.012569,-0.041727,0.038483,-0.015585
person_capacity,0.018539,0.091463,1.000000,-0.063531,0.055771,0.046826,0.070907,0.641390,-0.016742,-0.063936,0.073427,0.025229,-0.047466,-0.252243,0.012270,-0.006170
multi,-0.018551,-0.030984,-0.063531,1.000000,-0.472934,-0.024709,-0.000312,-0.019161,0.022684,0.007135,-0.036009,-0.016509,0.019317,0.050032,0.069881,-0.014141
biz,-0.020357,0.086842,0.055771,-0.472934,1.000000,-0.017531,-0.096956,-0.046903,-0.160681,-0.126562,0.168952,0.135079,0.015583,0.044644,-0.009279,0.003855
cleanliness_rating,-0.008189,0.041307,0.046826,-0.024709,-0.017531,1.000000,0.750295,-0.004457,-0.024183,-0.020695,0.041672,0.044145,-0.029794,-0.067969,0.265807,-0.007759
guest_satisfaction_overall,-0.016446,0.037163,0.070907,-0.000312,-0.096956,0.750295,1.000000,0.013659,-0.016962,0.003386,0.034607,0.033702,-0.015942,-0.063618,0.258607,-0.011637
bedrooms,0.024422,0.080990,0.641390,-0.019161,-0.046903,-0.004457,0.013659,1.000000,0.068413,0.017687,-0.020694,-0.050057,-0.019004,-0.086677,-0.013010,-0.011013
dist,0.144658,-0.112416,-0.016742,0.022684,-0.160681,-0.024183,-0.016962,0.068413,1.000000,0.084147,-0.572677,-0.456728,-0.007940,0.007058,-0.045165,0.019387
metro_dist,-0.020244,-0.052813,-0.063936,0.007135,-0.126562,-0.020695,0.003386,0.017687,0.084147,1.000000,-0.186509,-0.191793,-0.032691,-0.003648,-0.006273,0.006611


In [4]:
data_athens = data_athens.drop(['Unnamed: 0'], axis=1)

In [5]:
x = data_athens.drop('realSum', axis=1)
y = data_athens.realSum

In [6]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
count,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000,3960.000000
mean,3.697222,0.266919,0.384091,9.641667,95.041919,1.276515,1.801636,0.480442,5.753041,10.005139,0.001263,0.077778,0.430303,0.497727
std,1.284289,0.442406,0.486441,0.847343,8.391396,0.653012,0.952083,0.287191,4.541558,10.816142,0.035515,0.267855,0.495181,0.500058
min,2.000000,0.000000,0.000000,2.000000,20.000000,0.000000,0.070000,0.020000,1.040228,0.592757,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,0.000000,10.000000,94.000000,1.000000,1.120000,0.290000,3.050572,2.572603,0.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,10.000000,97.000000,1.000000,1.660000,0.420000,4.486832,6.345562,0.000000,0.000000,0.000000,0.000000
75%,4.000000,1.000000,1.000000,10.000000,100.000000,2.000000,2.300000,0.600000,6.614054,13.307919,0.000000,0.000000,1.000000,1.000000
max,6.000000,1.000000,1.000000,10.000000,100.000000,4.000000,6.220000,2.040000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     160.7
Date:                Mon, 06 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:02:23   Log-Likelihood:                -22164.
No. Observations:                3960   AIC:                         4.436e+04
Df Residuals:                    3945   BIC:                         4.445e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [8]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [9]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     160.7
Date:                Mon, 06 Mar 2023   Prob (F-statistic):               0.00
Time:                        17:02:23   Log-Likelihood:                -22164.
No. Observations:                3960   AIC:                         4.436e+04
Df Residuals:                    3945   BIC:                         4.445e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        149.3083      1.039    143.768      0.0

In [10]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

-38.80133139389858

In [11]:
model.score(x_train, y_train)

0.3632275358317346

In [12]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [13]:
mse(y_test,y_pred)

260136.58560952992

In [14]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

510.035866983421

In [15]:
##prediction on the train set
mse(y_train,y_pred_train)

4254.9274123214245

In [16]:
mae(y_test,y_pred)

53.392086664411444

In [17]:
R2=r2_score(y_test,y_pred)
R2

0.0093730967743898

In [18]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

-0.0012543182793716134

In [19]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.3628994847857311, Test -> 0.009240023801723951


In [20]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.34475313063510704, Test -> 0.007611757583439238


In [21]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 3960 rows and 14 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.01
The coefficient of determination for the train data is R2=1.00


In [22]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [0.28000258 0.27884321 0.35382906 0.25553947 0.35157599]
Score stats: 0.30 accuracy with a standard deviation of 0.04


In [23]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': 0.3178668824332441, 'Linear Regression': 0.3537717986230856, 'KNN': 0.28267146094187967}
